In [ ]:
# IMPORT MODULES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa
import librosa.feature
import librosa.display
import IPython.display as ipd
from pydub import AudioSegment
ms.use("seaborn-muted")
%matplotlib inline
import soundfile as sf
import sys
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, scale
from keras import models
from keras import layers
from keras.models import load_model
import tensorflow as tf
from scipy.stats import kurtosis
from scipy.stats import skew

Using TensorFlow backend.
/Users/fadrique/miniconda3/envs/IhFinalProject/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/fadrique/miniconda3/envs/IhFinalProject/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/fadrique/miniconda3/envs/IhFinalProject/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [ ]:
                                  # GET LIST OF FILES (SONG SPLICES) #

In [2]:
def getListOfFiles(dirName):

# Creates a list of files and sub directories #
    listOfFile = os.listdir(dirName)
    allFiles = list()
    
# Iterate over all the entries #
    for entry in listOfFile:
        
# Create full path
        fullPath = os.path.join(dirName, entry)
        
# If entry is a directory then get the list of files in this directory #    
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return [allf for allf in allFiles if allf.endswith(".wav")]

In [4]:
fullDataset = getListOfFiles("../DATA/SongSplits/")

In [ ]:
                                           # CREATE DF #

In [5]:
def create_final_df(input_array, sr=44100, n_fft=1024, hop_length=512):
    
# Split array into 20 splices #
    splitArray_songs = np.array_split(np.array(input_array),20)
    
    dictKeySong_ValueArray = {}
    s_c = []
    s_r = []
    s_f = []
    rms = []
    zcr = []
    chgrm = []
    mfcc = []
    
# loop over array of splices #
    for i in range(len(splitArray_songs)):
        for song in splitArray_songs[i]:

            y, sr = librosa.load(song, sr=sr)
            dictKeySong_ValueArray[song] = y
            
            s_c.append(librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel())
            s_r.append(librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel())
            s_f.append(librosa.onset.onset_strength(y=y, sr=sr).ravel())
            rms.append(librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel())
            zcr.append(librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel())
            chgrm.append(librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel())
            
# Append MFCC #
            mfcc_array = librosa.feature.mfcc(y, n_fft=n_fft, hop_length=hop_length, n_mfcc=15)
            mfcc.append(mfcc_array.ravel())
    
              
# Create DataFrame #                
    fullDS_df = pd.DataFrame(list(dictKeySong_ValueArray.items()))
    
    fullDS_df['Spectral_Centroid'] = s_c
    fullDS_df['Spectral_Rolloff'] = s_r
    fullDS_df['Spectral_flux'] = s_f
    fullDS_df['Rms'] = rms
    fullDS_df['Zcr'] = zcr
    fullDS_df['Chromagram'] = chgrm
    fullDS_df["Mfcc"] = mfcc
    
    fullDS_df = fullDS_df.rename(columns = {0: "Song_Name" , 1: "Song_Array_values"})
    
    fullDS_df.set_index('Song_Name')

    return fullDS_df

In [6]:
# Takes +- 40 mins to create DataFrame (Proceed with caution) #
final_df = create_final_df(fullDataset)
display(final_df.head())

,Song_Name,Song_Array_values,Spectral_Centroid,Spectral_Rolloff,Spectral_flux,Rms,Zcr,Chromagram,Mfcc
0,../DATA/SongSplits/blues.00000_1.wav,"[0.0073555857, 0.013357769, 0.016570134, 0.014...","[2245.0223442229703, 2111.621304098675, 2034.6...","[5167.96875, 4651.171875, 4565.0390625, 4091.3...","[0.0, 0.0, 0.0, 1.6228877, 0.87524116, 0.98208...","[0.025204595, 0.02760773, 0.027844246, 0.02634...","[0.0244140625, 0.0673828125, 0.08203125, 0.066...","[0.56689996, 0.37267882, 0.19090754, 0.2777658...","[-353.7237, -340.27148, -345.8821, -349.61725,..."
1,../DATA/SongSplits/blues.00000_10.wav,"[0.021448463, 0.0020720388, -0.027109465, -0.0...","[1643.672610943974, 1692.042400655203, 1489.24...","[3143.84765625, 3445.3125, 2928.515625, 3057.7...","[0.0, 0.0, 0.0, 0.81603897, 0.502655, 0.588014...","[0.12501433, 0.11911787, 0.113282606, 0.115545...","[0.0185546875, 0.0390625, 0.044921875, 0.04492...","[1.0, 0.5805376, 0.5022394, 0.4727841, 0.49316...","[-230.92538, -242.36787, -256.70215, -261.7970..."
2,../DATA/SongSplits/blues.00000_2.wav,"[-0.101408936, -0.1048613, -0.061488546, -0.01...","[2495.7311726977823, 2126.1061313586806, 2184....","[5727.83203125, 4909.5703125, 4952.63671875, 4...","[0.0, 0.0, 0.0, 1.6438422, 1.2619083, 1.257513...","[0.17098905, 0.15891351, 0.14206322, 0.1644201...","[0.025390625, 0.0693359375, 0.068359375, 0.046...","[0.57080966, 0.5964352, 0.8371001, 1.0, 1.0, 1...","[-186.20456, -207.50804, -218.2806, -210.48671..."
3,../DATA/SongSplits/blues.00000_3.wav,"[0.055520203, 0.046091527, 0.016172234, -0.003...","[1697.1847679213347, 1522.4794574243238, 1720....","[3488.37890625, 3273.046875, 3660.64453125, 39...","[0.0, 0.0, 0.0, 0.68196744, 0.5565205, 0.88246...","[0.062394187, 0.05248796, 0.038239926, 0.03434...","[0.0341796875, 0.060546875, 0.0498046875, 0.05...","[0.9838826, 0.47164562, 0.2351413, 0.5029442, ...","[-300.20505, -332.22653, -348.69525, -347.6219..."
4,../DATA/SongSplits/blues.00000_4.wav,"[-0.04392991, -0.0378538, -0.015621756, 0.0039...","[2092.5773769092357, 2160.9603109352215, 1540....","[4521.97265625, 5254.1015625, 3789.84375, 3660...","[0.0, 0.0, 0.0, 2.295959, 2.3464913, 2.691696,...","[0.09209392, 0.12053554, 0.13506459, 0.1944887...","[0.01953125, 0.0341796875, 0.04296875, 0.03906...","[1.0, 0.78802043, 0.6699747, 0.7824323, 0.3162...","[-274.49423, -257.0691, -249.3231, -227.73956,..."


In [ ]:
                                # GET INDEXES FOR DIFFERENT GENRES #

In [7]:
def findindex(genre):
    index_num = []
    for song in final_df["Song_Name"]:
        if genre in song:
            index = pd.Index(final_df['Song_Name']).get_loc(song)
            index_num.append(index)
    return index_num

In [8]:
blues_idx = findindex("blues")
classical_idx = findindex("classical")
country_idx = findindex("country")
disco_idx = findindex("disco")
hiphop_idx = findindex("hiphop")
jazz_idx = findindex("jazz")
metal_idx = findindex("metal")
pop_idx = findindex("pop")
reggae_idx = findindex("reggae")
rock_idx = findindex("rock")

In [ ]:
                                     # APPEND GENRE BY INDEX #

In [9]:
for i in blues_idx:
    final_df.loc[i, 'Genres'] = "blues"   
for i in classical_idx:
    final_df.loc[i, 'Genres'] = "classical"
for i in country_idx:
    final_df.loc[i, 'Genres'] = "country"    
for i in disco_idx:
    final_df.loc[i, 'Genres'] = "disco"    
for i in hiphop_idx:
    final_df.loc[i, 'Genres'] = "hiphop"   
for i in jazz_idx:
    final_df.loc[i, 'Genres'] = "jazz"    
for i in metal_idx:
    final_df.loc[i, 'Genres'] = "metal"    
for i in pop_idx:
    final_df.loc[i, 'Genres'] = "pop"    
for i in reggae_idx:
    final_df.loc[i, 'Genres'] = "reggae"   
for i in rock_idx:
    final_df.loc[i, 'Genres'] = "rock"

In [ ]:
                                       # GET FEATURE STATS #

In [10]:
def get_feat_stats(column, df = final_df):
    mean_lst = []
    std_lst = []
    kurt_lst = []
    skew_lst = []
    
    for i in range(len(df)):
        data_mean = final_df[column].loc[i].mean()
        data_std = final_df[column].loc[i].std()
        data_kurt = kurtosis(final_df[column].loc[i])
        data_skew = skew(final_df[column].loc[i])
        
        mean_lst.append(data_mean)
        std_lst.append(data_std)
        kurt_lst.append(data_kurt)
        skew_lst.append(data_skew)
        
    final_df['{}_Mean'.format(column)] = mean_lst
    final_df['{}_Std'.format(column)] = std_lst
    final_df['{}_Kurt'.format(column)] = kurt_lst
    final_df['{}_Skew'.format(column)] = skew_lst


In [ ]:
                             # CREATE FEATURE STAT COLUMNS FOR FINAL DF #

In [11]:
feat_lst = list(final_df)[2:9]

for i in feat_lst:
    get_feat_stats(i)


In [ ]:
                                       # DROP UNWANTED COLUMNS #

In [12]:
useless_cols = ["Song_Name", "Song_Array_values", 
                "Spectral_Centroid", "Spectral_Rolloff", 
                "Spectral_flux", "Rms", "Zcr", 
                "Chromagram", "Mfcc"]

In [13]:
final_df = final_df.drop(columns = useless_cols)
final_df.head()

,Genres,Spectral_Centroid_Mean,Spectral_Centroid_Std,Spectral_Centroid_Kurt,Spectral_Centroid_Skew,Spectral_Rolloff_Mean,Spectral_Rolloff_Std,Spectral_Rolloff_Kurt,Spectral_Rolloff_Skew,Spectral_flux_Mean,...,Zcr_Kurt,Zcr_Skew,Chromagram_Mean,Chromagram_Std,Chromagram_Kurt,Chromagram_Skew,Mfcc_Mean,Mfcc_Std,Mfcc_Kurt,Mfcc_Skew
0,blues,1641.999516,413.372535,9.483528,2.513262,3389.834595,1047.927955,3.396100,1.661935,1.319844,...,16.362803,3.291601,0.504586,0.292112,-1.124525,0.185129,-1.886445,92.105255,4.734897,-1.004656
1,blues,1853.959966,498.340301,3.700988,1.570845,4092.056274,1088.518603,0.201355,0.568283,1.310980,...,7.398224,2.246838,0.558865,0.281567,-1.122363,-0.002903,-0.750982,86.449547,4.595136,-1.073377
2,blues,1679.655326,440.306199,8.858783,2.431779,3473.425293,1090.428964,2.976799,1.558900,1.293437,...,9.311908,2.429808,0.488792,0.293371,-1.124648,0.316194,-2.085800,94.880653,4.434152,-0.917841
3,blues,1678.298601,341.277733,7.376823,1.422544,3605.316162,839.736075,2.650427,1.110475,1.291224,...,2.665304,1.070846,0.576497,0.279230,-1.142778,-0.078364,-1.849884,96.829147,4.869820,-1.264649
4,blues,1651.381950,308.995531,3.625571,0.931591,3565.240479,802.104401,4.520660,1.440690,1.245269,...,0.615136,0.837491,0.560615,0.293466,-1.092179,-0.152491,-1.579833,91.772133,5.390680,-1.432860


In [ ]:
                                     # REPLACE GENRE WITH VALUE #

In [14]:
final_df.replace(to_replace = "blues", value =0, inplace = True)
final_df.replace(to_replace = "classical", value =1, inplace = True)
final_df.replace(to_replace = "country", value =2, inplace = True)
final_df.replace(to_replace = "disco", value =3, inplace = True)
final_df.replace(to_replace = "hiphop", value =4, inplace = True)
final_df.replace(to_replace = "jazz", value =5, inplace = True)
final_df.replace(to_replace = "metal", value =6, inplace = True)
final_df.replace(to_replace = "pop", value =7, inplace = True)
final_df.replace(to_replace = "reggae", value =8, inplace = True)
final_df.replace(to_replace = "rock", value =9, inplace = True)

In [ ]:
                                   # EXPORT FINAL DATAFRAME AS PICKLE #

In [15]:
export_pickle = final_df.to_pickle("../DATA/PICKLES/Final_SongFeatures_Dataset.pkl") 